In [ ]:
import psycopg
from tqdm import tqdm 
from colorama import Style,Fore
import os
import json
from dotenv import load_dotenv
import time
import requests
import csv

load_dotenv()

In [2]:
def openJson(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def saveJson(path,data):
    with open(path, "w", encoding="utf-8") as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
       print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)

# Update DB with the new tables

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)


cur = conn.cursor()

cur.execute("""
    CREATE TABLE entites_spatiales (
        id_entite_spatiale TEXT PRIMARY KEY,
        label TEXT NOT NULL,
        latitude FLOAT NOT NULL,
        longitude FLOAT NOT NULL 
    )
""")

cur.execute("""
    CREATE TABLE entites_spatiales_videos (
        id_entite_spatiale TEXT REFERENCES entites_spatiales(id_entite_spatiale) ON DELETE CASCADE,
        id_video TEXT REFERENCES videos(id_video) ON DELETE CASCADE,
        PRIMARY KEY (id_video, id_entite_spatiale)
    )
""")

cur.execute("""
    CREATE TABLE entites_spatiales_chaines (
        id_entite_spatiale TEXT REFERENCES entites_spatiales(id_entite_spatiale) ON DELETE CASCADE,
        id_chaine TEXT REFERENCES chaines(id_chaine) ON DELETE CASCADE,
        PRIMARY KEY (id_chaine, id_entite_spatiale)
    )
""")


conn.commit()
cur.close()
conn.close()


# Fill the spacial_entities_videos table

## Prepare json

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)

cur = conn.cursor()
cur.execute("""
SELECT v.id_video,v.titre,v.description,v.tags 
FROM videos v JOIN chaines c ON v.id_chaine = c.id_chaine
WHERE c.pertinente = true;
"""
)
rows = cur.fetchall()
cur.close()
conn.close()

videos = []
for row in rows:
    id_video, titre, description, tags = row
    videos.append({
        "id_video": id_video,
        "titre": titre,
        "description": description,
        "tags": tags
    })

In [ ]:
len(videos)

In [ ]:
saveJson('./jsons/videosForSpacialAnalysis.json',videos)

## Process

### Functions

In [3]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = """
Tu es un extracteur d'entités géographiques.

À partir d’un texte donné, identifie toutes les localisations situées en France.

Ignore :
- les noms de pays (ex : "France"),
- les noms de personnes,
- les noms de chaînes YouTube, plateformes ou services numériques (ex : YouTube, Tipeee),
- les noms imaginaires, poétiques ou fictifs.

Retourne uniquement une **liste Python**, en **minuscules**, **sans doublons**, contenant **uniquement** des noms de lieux **réels** situés en **France**.

Aucune explication. **Donne uniquement le résultat au format requis.**
"""

user_template = "Contexte : {contexte}"

system_message = SystemMessagePromptTemplate.from_template(system_template)
user_message = HumanMessagePromptTemplate.from_template(user_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

In [4]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm_nvidia = ChatNVIDIA(
  model="meta/llama-3.1-8b-instruct",
  api_key=os.getenv('NVIDIA_API_KEY'), 
  temperature=0,
  top_p=0.7,
)

chain_nvidia =  chat_prompt | llm_nvidia

In [5]:
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(model="mistral-nemo:12b")
chain_ollama =  chat_prompt | llm_ollama

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
updatedVideosNew = openJson("./jsons/updatedVideosNew.json")
len(updatedVideosNew)

In [10]:
startFrom = len(updatedVideosNew)

def getContext(title,description,tags):
    videoContext = ''
    videoContext+=title
    videoContext+= '\n'+description
    if tags:
        videoContext += '\n'+ ', '.join(tags)
    return videoContext

def getLLMresponse_google(context,suffix):
    llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-06-17", temperature=0,api_key=os.getenv('GEMINI_API_KEY_'+suffix))
    chain_gemini =  chat_prompt | llm_gemini
    response = chain_gemini.invoke({'contexte':context})
    #print('response ',response)
    return response
  
def getLLMresponse_nvidia(context):
    response = chain_nvidia.invoke({'contexte':context})
    #print('response ',response)
    return response

def getLLMresponse_ollama(context):
    response = chain_ollama.invoke({'contexte':context})
    #print('response ',response)
    return response

def getSpacialEntities_google(context,suffix):
    response = getLLMresponse_google(context,suffix)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            Entities = []
            for e in entities:
                e_cleaned = e.lower().strip()
                Entities.append(e_cleaned)
            return Entities
    except:
        pass
    return None

def getSpacialEntities_nvidia(context):
    response = getLLMresponse_nvidia(context)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            Entities = []
            for e in entities:
                e_cleaned = e.lower().strip()
                Entities.append(e_cleaned)
            return Entities
    except:
        pass
    return None

def getSpacialEntities_ollama(context):
    response = getLLMresponse_ollama(context)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            Entities = []
            for e in entities:
                e_cleaned = e.lower().strip()
                Entities.append(e_cleaned)
            return Entities
    except:
        pass
    return None

def getGeocoding(entity):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": entity,
        "format": "json",
        "limit": 1,
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "geo-entity-extractor/1.0"
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data:
            lat = float(data[0]["lat"])
            lon = float(data[0]["lon"])
            country = data[0].get("address", {}).get("country", "unknown")
            return {
                'lat': lat,
                'lon': lon,
                'country': country
            }
    except Exception as e:
        print(f"Erreur pour l'entité '{entity}': {e}")
    
    return None

def runAll(jsonfile):
    videos = openJson(jsonfile)
    counter = 0
    MyAPIsuffix = ['MONO','NOUR','NOUR2008','TEXTRA','ZEG']
    #MyAPIsuffix = ['MONO_1','MONO_2','MONO_3','MONO_4','MONO_5']
    index = 0
    apiCounter = 0
    
    updatedVideosNew = openJson("./jsons/updatedVideosNew.json") # We open the old jsonfile so we continue from the video we stopped in.
    
    for video in tqdm(videos[startFrom:]):
        videoContext = getContext(video['titre'],video['description'],video['tags'])
        
        videoSpacialEntities = getSpacialEntities_google(videoContext,MyAPIsuffix[index])
        
        time.sleep(2)
        #videoSpacialEntities = getSpacialEntities_nvidia(videoContext)
        #videoSpacialEntities = getSpacialEntities_ollama(videoContext)
        
        #print("videoSpacialEntities  ",videoSpacialEntities)
        
        if videoSpacialEntities and len(videoSpacialEntities) > 0:
            output = []
            for ent in videoSpacialEntities:
                geocoding = getGeocoding(ent)
                if geocoding and geocoding['country']=='France':
                    geocoding['ent']=ent
                    output.append(geocoding)
            if len(output) >0 :
                video['output'] = output
                
        # Updating the new list
        updatedVideosNew.append(video)
        
        # Safe Saving 
        counter+= 1
        if counter == 20:
            saveJson("./jsons/updatedVideosNew.json",updatedVideosNew)
            counter =0
            
        # API Switching
        
        apiCounter +=1
        if apiCounter == 13:
            index+=1
            apiCounter = 0
            if index==len(MyAPIsuffix):
                print(Style.BRIGHT+Fore.BLUE+'\n sleep for 60s'+Style.RESET_ALL)
                time.sleep(60)
                index=0
            print(Style.BRIGHT+Fore.YELLOW+f'\n API KEY switched to {MyAPIsuffix[index]}'+Style.RESET_ALL)
            
        """
        apiCounter +=1
        if apiCounter == 35:
            print(Style.BRIGHT+Fore.BLUE+'\n sleep for 60s'+Style.RESET_ALL)
            time.sleep(60)
            apiCounter = 0
        """
        
    # Saving 
    saveJson("./jsons/updatedVideosNew.json",updatedVideosNew)

### Run on some samples

In [ ]:
testData = openJson('./jsons/test.json')
len(testData)

In [ ]:
MyAPIsuffix = ['MONO_1','MONO_2','MONO_3','MONO_4','MONO_5']
index = 0
apiCounter = 0

for video in tqdm(testData):
    output = []
    context = getContext(video['titre'],video['description'],video['tags'])
    entities = getSpacialEntities_google(context,MyAPIsuffix[index])
    #entities = getSpacialEntities_ollama(context)
    print('entities :',entities)
    if entities:
        for ent in entities:
            result = {'ent':ent}
            geoCode = getGeocoding(ent)
            if geoCode:
                result.update(geoCode)
            output.append(result)
            #print('output  :',output)
    video['output'] = output
            
    # API Switching
     
    apiCounter +=1
    if apiCounter == 13:
        index+=1
        apiCounter = 0
        if index==5:
            print(Style.BRIGHT+Fore.BLUE+'\n sleep for 60s'+Style.RESET_ALL)
            time.sleep(60)
            index=0
        print(Style.BRIGHT+Fore.YELLOW+f'\n API KEY switched to {MyAPIsuffix[index]}'+Style.RESET_ALL)
     
    

In [ ]:
saveJson('./jsons/testResults.json',testData)

### Plot in map an Entity

In [10]:
import folium

location_data = {
        "ent": "brian",
        "lat": 45.5913283,
        "lon": 12.8142248,
      }
map_obj = folium.Map(location=[location_data["lat"], location_data["lon"]], zoom_start=13)

folium.Marker(
    [location_data["lat"], location_data["lon"]],
    popup=location_data["ent"],
    tooltip=location_data["ent"]
).add_to(map_obj)

map_obj.save("map_janze.html")


### Run All

In [11]:
runAll("./jsons/videosForSpacialAnalysis.json")

 17%|█▋        | 3510/20563 [3:32:45<97:48:38, 20.65s/it]


 API KEY switched to MONO


 17%|█▋        | 3520/20563 [3:33:09<14:54:16,  3.15s/it]


 json saved


 17%|█▋        | 3523/20563 [3:33:17<12:54:28,  2.73s/it]

...


 json saved

 sleep for 60s


 24%|██▍       | 4940/20563 [4:58:50<90:31:38, 20.86s/it]


 API KEY switched to MONO


 24%|██▍       | 4952/20563 [4:59:19<11:24:19,  2.63s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 46
}
].
 24%|██▍       | 4952/20563 [4:59:22<15:43:44,  3.63s/it]


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
]

## Old Process

In [ ]:

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = """
Tu es un extracteur d'entités géographiques françaises.
À partir d’un texte donné, identifie uniquement les **villes**, **communes** situés en France.
Ne prends **pas** en compte :
- les noms de pays (ex: "France"),
- les noms de personnes,
- les noms de chaînes YouTube, de plateformes (ex: YouTube, Tipeee),
- les noms imaginaires ou poétiques.

Retourne une **liste Python**, en minuscules, sans doublons, contenant uniquement des noms de lieux réels en France.
Pas d'explication, donner la reponse en format string.

**Les noms extraits doivent être en français**
**Pas d'explication juste la liste**
"""

user_template = "Contexte : {contexte}"

system_message = SystemMessagePromptTemplate.from_template(system_template)
user_message = HumanMessagePromptTemplate.from_template(user_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

In [29]:
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(model="llama3.1:8b")
chain_ollama =  chat_prompt | llm_ollama

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm_nvidia = ChatNVIDIA(
  model="meta/llama-3.1-8b-instruct",
  api_key=os.getenv('NVIDIA_API_KEY'), 
  temperature=0,
  top_p=0.7,
)

chain_nvidia =  chat_prompt | llm_nvidia

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

{
    id_video = '',
    titre : '',
    description:'',
    tags:''
    +
    output : [
            {
            ent : Ent1
            lat :
            lon : },
            {
            ent : Ent2
            lat :
            lon : },
        ...
    ]
}

In [ ]:
updatedVideos = openJson("./jsons/updatedVideos.json")
len(updatedVideos)

In [ ]:
startFrom = len(updatedVideos)

def getContext(title,description,tags):
    videoContext = ''
    videoContext+=title
    videoContext+= '\n'+description
    if tags:
        videoContext += '\n'+ ', '.join(tags)
    return videoContext

def getEntityVerification(entity,csvfile,column):
    with open(csvfile, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row[column].strip().lower() == entity:
                return True
    return False

def getLLMresponse_google(context,suffix):
    llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-06-17", temperature=0,api_key=os.getenv('GEMINI_API_KEY_'+suffix))
    chain_gemini =  chat_prompt | llm_gemini
    response = chain_gemini.invoke({'contexte':context})
    #print('response ',response)
    return response
   
def getLLMresponse_nvidia(context):
    response = chain_nvidia.invoke({'contexte':context})
    #print('response ',response)
    return response
  
def getSpacialEntities_google(context,suffix):
    response = getLLMresponse_google(context,suffix)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            Entities = []
            for e in entities:
                e_cleaned = e.lower().strip()
                if getEntityVerification(e_cleaned,'./csvs/v_commune_2025.csv','NCCENR'):
                    Entities.append(e_cleaned)
            return Entities
    except:
        pass
    return []

def getSpacialEntities_nvidia(context):
    response = getLLMresponse_nvidia(context)
    
    try:
        entities = eval(response.content.strip())
        if isinstance(entities, list):
            Entities = []
            for e in entities:
                e_cleaned = e.lower().strip()
                if getEntityVerification(e_cleaned,'./csvs/v_commune_2025.csv','NCCENR'):
                    Entities.append(e_cleaned)
            return Entities
    except:
        pass
    return []

def getGeocoding(entity):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": entity + ", France",
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "geo-entity-extractor/1.0"
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        if data:
            lat = float(data[0]["lat"])
            lon = float(data[0]["lon"])
            return {'lat':lat,
                    'lon':lon}
    except Exception as e:
        print(f"Erreur pour l'entité '{entity}': {e}")
    
    return None

def runAll(jsonfile):
    videos = openJson(jsonfile)
    counter = 0
    MyAPIsuffix = ['MONO','NOUR','NOUR2008','TEXTRA','ZEG']
    index = 0
    apiCounter = 0
    
    updatedVideos = openJson("./jsons/updatedVideos.json") # We open the old jsonfile so we continue from the video we stopped in.
    
    for video in tqdm(videos[startFrom:]):
        videoContext = getContext(video['titre'],video['description'],video['tags'])
        
        videoSpacialEntities = getSpacialEntities_google(videoContext,MyAPIsuffix[index])
        
        #videoSpacialEntities = getSpacialEntities_nvidia(videoContext)
        
        #print("videoSpacialEntities  ",videoSpacialEntities)
        if len(videoSpacialEntities) > 0:
            output = []
            for ent in videoSpacialEntities:
                geocoding = getGeocoding(ent)
                if geocoding :
                    geocoding['ent']=ent
                    output.append(geocoding)
            if len(output) >0 :
                video['output'] = output
                
        # Updating the new list
        updatedVideos.append(video)
        
        # Safe Saving 
        counter+= 1
        if counter == 100:
            saveJson("./jsons/updatedVideos.json",updatedVideos)
            counter =0
            
        # API Switching
        
        apiCounter +=1
        if apiCounter == 13:
            index+=1
            apiCounter = 0
            if index==5:
                print(Style.BRIGHT+Fore.BLUE+'\n sleep for 60s'+Style.RESET_ALL)
                time.sleep(60)
                index=0
            print(Style.BRIGHT+Fore.YELLOW+f'\n API KEY switched to {MyAPIsuffix[index]}'+Style.RESET_ALL)
            
        """
        apiCounter +=1
        if apiCounter == 35:
            print(Style.BRIGHT+Fore.BLUE+'\n sleep for 60s'+Style.RESET_ALL)
            time.sleep(60)
            apiCounter = 0
        """
        
    # Saving 
    saveJson("./jsons/updatedVideos.json",updatedVideos)

- Test

In [11]:
title = "Ils vivent dans une maison bâtie avec des déchets"
description = "\"Elle nous protège, elle nous nourrit, elle nous réchauffe, elle nous offre tous nos besoins primaires.\"\n\nPendant ce temps-là à Biras, en Dordogne, Pauline, Benjamin et Noéha vivent dans cette maison enterrée, autonome en énergie et bâtie avec des déchets. Visite de leur earthship.\n\n————————————— \n▶︎ Retrouvez la vidéo sur le site de Brut https://www.brut.media/fr/science-and-technology/ils-vivent-dans-une-maison-batie-avec-des-dechets-cebe8641-ba94-4e76-843a-10b58e4a35fa\n▶ 📲 sur l’appli Brut (iOS) : https://apple.co/2UY7gNH \n▶ 📲 sur l’appli Brut (Android) : https://play.google.com/store/apps/details?id=media.brut.brut \n👉 Abonnez-vous à la newsletter myBrut : https://bit.ly/2JhQ5pP\n▶ Pour ne rien louper des vidéos Brut, n’hésitez pas à vous abonner ➞ https://www.youtube.com/channel/UCSKdvgqdnj72_SLggp7BDTg/?sub_confirmation=1 et à activer la cloche 🔔"
tags =  [
      "brut",
      "déchet",
      "maison",
      "construction",
      "poubelle",
      "verre",
      "recyclage"
    ]

videoTestContexte = getContext(title, description, tags)

#print(videoTestContexte)


In [31]:
# Exemple de texte avec des noms de lieux
texte_contenu = """
Lors de mon voyage en Provence, j’ai visité Marseille, le quartier du Panier, Aix-en-Provence 
et un petit village appelé Eygalières. Ensuite, nous sommes allés à Nice et dans le Vieux-Nice.
"""

In [ ]:
getSpacialEntities_google(videoTestContexte,'MONO_1')

In [ ]:
getLLMresponse_ollama(texte_contenu)

In [ ]:
getSpacialEntities_nvidia(videoTestContexte)

In [ ]:
getEntityVerification('biras','./csvs/v_commune_2025.csv','NCCENR')

In [ ]:
getGeocoding('provence')

- Run on All

In [ ]:
runAll("./jsons/videosForSpacialAnalysis.json")

## Videos of non pertinents channels

In [ ]:
updatedVideos = openJson("./jsons/updatedVideos.json")
len(updatedVideos)

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)


cur = conn.cursor()

cur.execute("""
    SELECT v.id_video
    FROM videos v join chaines c
    ON v.id_chaine = c.id_chaine
    WHERE c.pertinente= false ;
""")

rows = cur.fetchall()
cur.close()
conn.close()

ids = [row[0] for row in rows]
cleaned = []
len(ids)


In [ ]:
for video in updatedVideos:
    if video['id_video'] not in ids:
            cleaned.append(video)

In [ ]:
len(cleaned)

In [ ]:
saveJson("./jsons/updatedVideos.json",cleaned)

In [ ]:
videosForSpacialAnalysis = openJson("./jsons/updatedVideos.json")
len(updatedVideos)

### Plot coordinates

In [ ]:
import folium

location_data = {
        "lat": 49.5532646,
        "lon": 2.9392577,
        "ent": "ville"
      }

map_obj = folium.Map(location=[location_data["lat"], location_data["lon"]], zoom_start=13)

folium.Marker(
    [location_data["lat"], location_data["lon"]],
    popup=location_data["ent"],
    tooltip=location_data["ent"]
).add_to(map_obj)

map_obj.save("map_janze.html")
